In [236]:
import numpy as np;
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from scipy.optimize import curve_fit
import glob
import os
import sys
import matplotlib.backends.backend_pdf
import matplotlib.gridspec as gridspec
from scipy.stats import norm
from scipy.stats import lognorm
from scipy import linalg
import pprint
import re
plt.rc('font', family='serif')
from matplotlib import rc
import pandas as pd
from astropy import constants as const
from astropy import units as units
from IPython.display import Markdown as md
import sympy
from sympy import integrate, diff, sqrt, cos, sin, pi, exp, log
from sympy.abc import * 
i = sqrt(-1)
import numpy as np
import sympy.printing as printing
latp = printing.latex
hbar = sympy.symbols("hbar", real=True)
import mpmath 
import plotly.express as px
hbar

hbar

Where $B$ is the spectral radiance and $f$ is the frequency, we have Planck's law of Blackbody Radiation. We will be using CGS units.

In [244]:
def B(T, l, number = False, dps=50):
    if number:
        mpmath.mp.dps = dps
        k_B = const.k_B.value
        h = const.h.value
        c = const.c.value
        T = mpmath.mpmathify(T)
        return 2*h* ( ( c / l )**3 ) / (
            c**2 * (mpmath.exp(h * c/ (l * k_B * T)) - 1))
    else:
        k_B, h, c = sympy.symbols("k_B h c")
        return 2*h* ( ( c / l )**3 ) / (
            c**2 * (exp(h * c/ (l * k_B * T)) - 1))
md("We have the Planck Blackbody Law" + "\\begin{equation} B_l = " + latp(B(T,c/v))
   + "\\end{equation}")

We have the Planck Blackbody Law\begin{equation} B_l = \frac{2 h v^{3}}{c^{2} \left(e^{\frac{h v}{T k_{B}}} - 1\right)}\end{equation}

In [263]:
from sympy.plotting import plot
def plot_planck(T):
    #plt.figure(figsize=(5,5))
    plt.figure(dpi = 800)
    wavelengths = np.linspace(350 * 10**-9, 1600 * 10**-9, 10**5)
    radiances = [B(T, l, True) for l in wavelengths]
    plt.scatter(wavelengths, radiances)
    plt.xlabel("Wavelength (m)")
    plt.grid()
    plt.ylabel("Spectral Radiance (W Hz$^{-1}$ sr$^{-1}$ m$^2$)") 
    plt.title(str(T) + " Kelvin Ideal Blackbody")
    '''
    plot(b_fun, xlim = (350 * 10**-9, 1600 * 10**-9), ylim=(.9,10),
        adaptive = False, nb_of_points = 10**8, yscale="log", xlabel = "Wavelength (nm)",
        title = "Planck Function for Blackbody at " + str(T) + " Kelvin",
        ylabel = "Spectral Radiance (W Hz$^{-1}$ sr$^{-1}$ m$^2$)")
    '''
    return plt

In [ ]:
plot_planck(300)

In [ ]:
plot_planck(1000)

In [ ]:
plot_planck(3000)

In [ ]:
plot_planck(5000)

In [259]:
temp_list = [300, 1000, 3000, 5000]
for T in temp_list:
    peak = (3 * units.mm / T).to(units.m).value
    print(mpmath.log10(peak), mpmath.log10(B(T, peak, True)))

-4.9999999999999999644733850823019973008295631257525 -11.480132488738328539155182878367842691858105964728
-5.5228787452803375517024728445662672420376910518749 -9.9114962528973159081789007475600640111092417400786
-6.0000000000000000196526453297206160616919506824173 -8.4801324887383286630048454686219256681457103788396
-6.2218487496163563871438785317410077257236319367645 -7.8145862398892592686797972725455481807151190300996
